In [2]:
'''
This notebook is a standalone script. This requires different packages which is not mentioned in the kg-rag requirements.txt file.
This notebook serves as a proof of concept that KG-RAG can be extended to other node types to answer more versatile biomedical questions.
'''

from llama_index.core import Settings
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding

from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

from pinecone import Pinecone
from dotenv import load_dotenv, find_dotenv
import os
import json

from joblib import Memory
from tenacity import retry, stop_after_attempt, wait_random_exponential
import requests
import pandas as pd
import numpy as np
import ast
from sklearn.metrics.pairwise import cosine_similarity
import time
import sys

from IPython.display import clear_output


In [4]:
INDEX_NAME = 'spoke-nodes'
EMBEDDING_MODEL = 'text-embedding-ada-002'
CHAT_MODEL = 'gpt-35-turbo'
SENTENCE_EMBEDDING_MODEL_FOR_CONTEXT_RETRIEVAL = 'pritamdeka/S-PubMedBert-MS-MARCO'

chat_memory = Memory('cachegpt', verbose=0)
embed_memory = Memory('cache_embedding', verbose=0)

SPOKE_API_URI = 'https://spoke.rbvi.ucsf.edu'

KG_RAG_PARAMS = {
    'CONTEXT_VOLUME' : 150,
    'QUESTION_VS_CONTEXT_SIMILARITY_PERCENTILE_THRESHOLD' : 75,
    'QUESTION_VS_CONTEXT_MINIMUM_SIMILARITY' : 0.5,
}

DEFAULT_SPOKE_API_PARAMS = {
    'cutoff_Compound_max_phase' : 3,
    'cutoff_Protein_source' : ['SwissProt'],
    'cutoff_DaG_diseases_sources' : ['knowledge', 'experiments', 'textmining'],
    'cutoff_DaG_textmining' : 3,
    'cutoff_CtD_phase' : 3,
    'cutoff_PiP_confidence' : 0.7,
    'cutoff_ACTeG_level' : ['Low', 'Medium', 'High'],
    'depth' : 1,
    'cutoff_DpL_average_prevalence' : 0.001
}

In [5]:
BIOMEDICAL_ENTITY_EXTRACTION_SYSTEM_PROMPT = '''
  You are an expert biomedical name entity extractor from a sentence. 
  You are able to extract entities such as Diseases, Genes, Variants, Proteins, Symptoms, Side effects, Organisms, Pathways, Compounds, Reactions. 
  Report your response as a JSON in the following format:
  {"Entities": <List of extracted biomedical entities from the user input>}
  IMPORTANT : Report the names mentioned in the user input, not the criteria they belong to. 
  
  Example 1:
  User input : What are the proteins encoded by PINK1?
  Response: {"Entities" : ["PINK1"]}
  
  Example 2:
  User input: Other than Parkinson's disease, what diseases do SNCA connect to?
  Response: {"Entities": ["Parkinson's disease", "SNCA"]}
'''


KG_RAG_BASED_TEXT_GENERATION_SYSTEM_PROMPT = '''
  You are an expert biomedical researcher. 
  For answering the Question at the end with brevity, you need to first read the Context provided. 
  Then give your final answer briefly, by citing the Provenance information from the context. 
  You can find Provenance from the Context statement 'Provenance of this association is <Provenance>'. 
  Do not forget to cite the Provenance information. Note that, if Provenance is 'GWAS' report it as 'GWAS Catalog'. 
  If Provenance is 'DISEASES' report it as 'DISEASES database - https://diseases.jensenlab.org'. 
  Additionally, when providing drug or medication suggestions, give maximum information available and then advise the user to seek guidance from a healthcare professional as a precautionary measure.
'''


In [20]:
def load_sentence_transformer(sentence_embedding_model):
    return SentenceTransformerEmbeddings(model_name=sentence_embedding_model)
    
def biomedical_entity_extractor(text, model):
    prompt_updated = BIOMEDICAL_ENTITY_EXTRACTION_SYSTEM_PROMPT + "\n" + "Sentence : " + text
    resp = get_GPT_response(text, BIOMEDICAL_ENTITY_EXTRACTION_SYSTEM_PROMPT, model, temperature=0)
    try:
        entity_dict = json.loads(resp)
        return entity_dict["Entities"]
    except:
        return None

@embed_memory.cache
def openai_embedding(embed_model, text):
    node_embedding = embed_model.get_text_embedding(text)
    return node_embedding 


@chat_memory.cache
def get_GPT_response(instruction, system_prompt, chat_model_id, temperature=0.3):
    return fetch_GPT_response(instruction, system_prompt, chat_model_id, temperature=temperature)

@retry(wait=wait_random_exponential(min=10, max=30), stop=stop_after_attempt(5))
def fetch_GPT_response(instruction, system_prompt, chat_model_id, temperature):
    response = client.chat.completions.create(        
        temperature=temperature,
        model=chat_model_id,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": instruction}
        ]
    )
    if response.choices:
        return response.choices[0].message.content
    else:
        return 'Unexpected response'

def retrieve_node(query, embed_model, k=1):
    query_embedding = embed_model.get_text_embedding(query)
    retrieved_doc = pinecone_index.query(
        vector=query_embedding, 
        top_k=k, 
        include_metadata=True
    )
    node_name = list(map(lambda x:json.loads(x['metadata']['_node_content'])['text'], retrieved_doc['matches']))
    node_type = list(map(lambda x:x['metadata']['node_type'], retrieved_doc['matches']))
    return node_name, node_type


def get_spoke_api_resp(base_uri, end_point, params=None):
    uri = base_uri + end_point
    if params:
        return requests.get(uri, params=params)
    else:
        return requests.get(uri)

@retry(wait=wait_random_exponential(min=10, max=30), stop=stop_after_attempt(5))
def get_context_using_spoke_api(node_value, node_type, search_depth):
    type_end_point = "/api/v1/types"
    result = get_spoke_api_resp(SPOKE_API_URI, type_end_point)
    data_spoke_types = result.json()
    node_types = list(data_spoke_types["nodes"].keys())
    edge_types = list(data_spoke_types["edges"].keys())
    node_types_to_remove = ["DatabaseTimestamp", "Version"]
    filtered_node_types = [node_type for node_type in node_types if node_type not in node_types_to_remove]
    api_params = {
        'node_filters' : filtered_node_types,
        'edge_filters': edge_types,
        'cutoff_Compound_max_phase': DEFAULT_SPOKE_API_PARAMS['cutoff_Compound_max_phase'],
        'cutoff_Protein_source': DEFAULT_SPOKE_API_PARAMS['cutoff_Protein_source'],
        'cutoff_DaG_diseases_sources': DEFAULT_SPOKE_API_PARAMS['cutoff_DaG_diseases_sources'],
        'cutoff_DaG_textmining': DEFAULT_SPOKE_API_PARAMS['cutoff_DaG_textmining'],
        'cutoff_CtD_phase': DEFAULT_SPOKE_API_PARAMS['cutoff_CtD_phase'],
        'cutoff_PiP_confidence': DEFAULT_SPOKE_API_PARAMS['cutoff_PiP_confidence'],
        'cutoff_ACTeG_level': DEFAULT_SPOKE_API_PARAMS['cutoff_ACTeG_level'],
        'cutoff_DpL_average_prevalence': DEFAULT_SPOKE_API_PARAMS['cutoff_DpL_average_prevalence'],
        'depth' : search_depth
    }
    attribute = "name"
    nbr_end_point = "/api/v1/neighborhood/{}/{}/{}".format(node_type, attribute, node_value)
    result = get_spoke_api_resp(SPOKE_API_URI, nbr_end_point, params=api_params)
    node_context = result.json()
    if len(node_context) == 0: 
        return None, None
    nbr_nodes = []
    nbr_edges = []
    for item in node_context:
        if "_" not in item["data"]["neo4j_type"]:
            try:
                if item["data"]["neo4j_type"] == "Protein":
                    nbr_nodes.append((item["data"]["neo4j_type"], item["data"]["id"], item["data"]["properties"]["description"]))
                else:
                    nbr_nodes.append((item["data"]["neo4j_type"], item["data"]["id"], item["data"]["properties"]["name"]))
            except:
                nbr_nodes.append((item["data"]["neo4j_type"], item["data"]["id"], item["data"]["properties"]["identifier"]))
        elif "_" in item["data"]["neo4j_type"]:
            try:
                provenance = ", ".join(item["data"]["properties"]["sources"])
            except:
                try:
                    provenance = item["data"]["properties"]["source"]
                    if isinstance(provenance, list):
                        provenance = ", ".join(provenance)                    
                except:
                    try:                    
                        preprint_list = ast.literal_eval(item["data"]["properties"]["preprint_list"])
                        if len(preprint_list) > 0:                                                    
                            provenance = ", ".join(preprint_list)
                        else:
                            pmid_list = ast.literal_eval(item["data"]["properties"]["pmid_list"])
                            pmid_list = map(lambda x:"pubmedId:"+x, pmid_list)
                            if len(pmid_list) > 0:
                                provenance = ", ".join(pmid_list)
                            else:
                                provenance = "Based on data from Institute For Systems Biology (ISB)"
                    except:                                
                        provenance = "SPOKE-KG"     
            try:
                evidence = item["data"]["properties"]
            except:
                evidence = None
            nbr_edges.append((item["data"]["source"], item["data"]["neo4j_type"], item["data"]["target"], provenance, evidence))
    nbr_nodes_df = pd.DataFrame(nbr_nodes, columns=["node_type", "node_id", "node_name"])
    nbr_edges_df = pd.DataFrame(nbr_edges, columns=["source", "edge_type", "target", "provenance", "evidence"])
    merge_1 = pd.merge(nbr_edges_df, nbr_nodes_df, left_on="source", right_on="node_id").drop("node_id", axis=1)
    merge_1.loc[:,"node_name"] = merge_1.node_type + " " + merge_1.node_name
    merge_1.drop(["source", "node_type"], axis=1, inplace=True)
    merge_1 = merge_1.rename(columns={"node_name":"source"})
    merge_2 = pd.merge(merge_1, nbr_nodes_df, left_on="target", right_on="node_id").drop("node_id", axis=1)
    merge_2.loc[:,"node_name"] = merge_2.node_type + " " + merge_2.node_name
    merge_2.drop(["target", "node_type"], axis=1, inplace=True)
    merge_2 = merge_2.rename(columns={"node_name":"target"})
    merge_2 = merge_2[["source", "edge_type", "target", "provenance", "evidence"]]
    merge_2.loc[:, "predicate"] = merge_2.edge_type.apply(lambda x:x.split("_")[0])
    merge_2.loc[:, "context"] =  merge_2.source + " " + merge_2.predicate.str.lower() + " " + merge_2.target + " and Provenance of this association is " + merge_2.provenance + ". "
    context = merge_2.context.str.cat(sep=' ')
    # context += node_value + " has a " + node_context[0]["data"]["properties"]["source"] + " identifier of " + node_context[0]["data"]["properties"]["identifier"] + " and Provenance of this is from " + node_context[0]["data"]["properties"]["source"] + ". "
    return context, merge_2


def get_pruned_context(query, entities, embed_model, search_depth, edge_evidence, context_sim_threshold, context_sim_min_threshold, context_volume):
    if entities:
        nodes = []
        node_types = []
        for entity in entities:
            node_name_list, node_type_list = retrieve_node(entity, openai_embed_model, k=2)
            nodes.extend(node_name_list)
            node_types.extend(node_type_list)
        max_number_of_high_similarity_context_per_node = int(context_volume/len(nodes))
        # query_embedding = openai_embedding(embed_model, query)
        query_embedding = embed_model.embed_query(query)
        node_context_extracted = ''
        for node_idx, node in enumerate(nodes):
            node_context, context_table = get_context_using_spoke_api(
                node_value=node,
                node_type = node_types[node_idx],
                search_depth=search_depth
            )        
            if not node_context:
                node_context_extracted += f'No context related to {node} available in SPOKE'
                continue
            context_table.loc[:, 'context'] = context_table.context.apply(lambda x:x.strip())
            node_context_list = node_context.split(". ")
            node_context_list = list(map(lambda x:x.strip(), node_context_list))
            if len(node_context_list) < 4500000000000:
                node_context_list = ['empty string' if item == '' else item for item in node_context_list]
                # node_context_embeddings = [openai_embedding(embed_model, node_context) for node_context in node_context_list]            
                node_context_embeddings = embed_model.embed_documents(node_context_list)
                similarities = [cosine_similarity(np.array(query_embedding).reshape(1, -1), np.array(node_context_embedding).reshape(1, -1)) for node_context_embedding in node_context_embeddings]
                similarities = sorted([(e, i) for i, e in enumerate(similarities)], reverse=True)
                percentile_threshold = np.percentile([s[0] for s in similarities], context_sim_threshold)
                high_similarity_indices = [s[1] for s in similarities if s[0] > percentile_threshold and s[0] > context_sim_min_threshold]
                if len(high_similarity_indices) > max_number_of_high_similarity_context_per_node:
                    high_similarity_indices = high_similarity_indices[:max_number_of_high_similarity_context_per_node]
                high_similarity_context = [node_context_list[index].strip() + '.' for index in high_similarity_indices]
                if edge_evidence:                
                    # if len(high_similarity_context) > 1:
                    #     high_similarity_context = list(map(lambda x:x+'.', high_similarity_context))
                    context_table = context_table[context_table.context.isin(high_similarity_context)]
                    context_table.loc[:, "context"] =  context_table.source + " " + context_table.predicate.str.lower() + " " + context_table.target + " and Provenance of this association is " + context_table.provenance + " and attributes associated with this association is in the following JSON format:\n " + context_table.evidence.astype('str') + "\n\n"                
                    node_context_extracted += context_table.context.str.cat(sep=' ')                      
                else:
                    node_context_extracted += ". ".join(high_similarity_context)
                    node_context_extracted += ". "
            else:
                if edge_evidence:
                    context_table.loc[:, "context"] =  context_table.source + " " + context_table.predicate.str.lower() + " " + context_table.target + " and Provenance of this association is " + context_table.provenance + " and attributes associated with this association is in the following JSON format:\n " + context_table.evidence.astype('str') + "\n\n"                
                    node_context_extracted += context_table.context.str.cat(sep=' ')
                else:
                    node_context_extracted += ". ".join(node_context_list)                
        return node_context_extracted
    else:
        return 'No biomedical entities detected in the given query.'

def stream_out(output):
    CHUNK_SIZE = int(round(len(output)/50))
    SLEEP_TIME = 0.1
    for i in range(0, len(output), CHUNK_SIZE):
        print(output[i:i+CHUNK_SIZE], end='')
        sys.stdout.flush()
        time.sleep(SLEEP_TIME)
    print("\n")

def run(query):
    entity = biomedical_entity_extractor(query, CHAT_MODEL)
    kg_cntxt = get_pruned_context(
        query, 
        entity,
        bert_embed_model,
        1,
        True,
        KG_RAG_PARAMS['QUESTION_VS_CONTEXT_SIMILARITY_PERCENTILE_THRESHOLD'],
        KG_RAG_PARAMS['QUESTION_VS_CONTEXT_MINIMUM_SIMILARITY'],
        KG_RAG_PARAMS['CONTEXT_VOLUME']
    )
    enriched_prompt = 'Context: '+ kg_cntxt + '\n' + 'Question: ' + query 
    llm_response = get_GPT_response(
        enriched_prompt, 
        KG_RAG_BASED_TEXT_GENERATION_SYSTEM_PROMPT, 
        'gpt-35-turbo-16k', 
        temperature=0.1
    )
    stream_out(llm_response)

In [7]:
load_dotenv(os.path.join(os.path.expanduser('~'), '.gpt_config.env'))
api_key = os.environ.get('API_KEY')
azure_endpoint = os.environ.get('RESOURCE_ENDPOINT')
api_version = os.environ.get('API_VERSION')

load_dotenv(os.path.join(os.path.expanduser('~'), '.pinecone_config.env'))
pinecone_api_key = os.environ.get('PINECONE_API_KEY')


if os.environ.get('RESOURCE_ENDPOINT'):
    from openai import AzureOpenAI    
    client = AzureOpenAI(
      api_key = os.environ.get('API_KEY'),  
      api_version = os.environ.get('API_VERSION'),
      azure_endpoint = os.environ.get('RESOURCE_ENDPOINT')
    )
else:
    from openai import OpenAI
    client = OpenAI(api_key = os.environ.get('API_KEY'))

openai_embed_model = AzureOpenAIEmbedding(
    model=EMBEDDING_MODEL,
    deployment_name=EMBEDDING_MODEL,
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

bert_embed_model = load_sentence_transformer(SENTENCE_EMBEDDING_MODEL_FOR_CONTEXT_RETRIEVAL)

Settings.embed_model = openai_embed_model

pc = Pinecone(api_key=pinecone_api_key)

pinecone_index = pc.Index(INDEX_NAME)

/root/anaconda3/envs/kg_rag_interative/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [21]:

query = "What protein does PAH gene encode?"
run(query)


The PAH gene encodes the protein Phenylalanine-4-hydroxylase (PAH). [Provenance: UniProt]



In [25]:

query = "What reactions do phenylalanine-4-hydroxylase catalyse?"
run(query)


Phenylalanine-4-hydroxylase (PAH) catalyzes the following reactions:
1. Reaction: L-phenylalanine + tetrahydrobiopterin + oxygen <=> dihydrobiopterin + L-tyrosine + H2O
   Provenance: KEGG database - https://www.genome.jp/kegg/

2. Reaction: L-phenylalanine + tetrahydrobiopterin + oxygen <=> L-tyrosine + 4a-hydroxytetrahydrobiopterin
   Provenance: KEGG database - https://www.genome.jp/kegg/

3. Reaction: L-phenylalanine + tetrahydropteridine + oxygen <=> L-tyrosine + 4a-hydroxy-5,6,7,8-tetrahydropteridine
   Provenance: KEGG database - https://www.genome.jp/kegg/



In [28]:

query = "Does Balaglitazone downregulate any genes?"
run(query)


Yes, Balaglitazone downregulates several genes. The genes that are downregulated by Balaglitazone include ADI1, ACOT9, IFRD2, CCNB1, H2BC12, RPS5, DHRS7, UGDH, KDELR2, WASHC4, LYPLA1, PRPF4, GOLT1B, HSPB1, AKR7A2, BAG3, GNAS, SOX4, DUSP6, CDC25B, FAT1, PRSS23, GRN, CANT1, HOXA10, IER3, PXN, CTNND1, BNIP3, HDAC6, KDM3A, SPDEF, ARID5B, ST6GALNAC2, DUSP4, TRAPPC6A, EPB41L2, IQGAP1, ASAH1, PHGDH, RGS2, FOXO4, TIMELESS, TCEA2, CD44, CDH3, DNAJA3, CDKN1B, PIK3R3, C2CD5, GLRX, HAT1, PTPRK, H2BC5, AKR7A2, ITFG1, KIFBP, S100A6, TRIM2, SLC2A6, ALDOC, ST3GAL5, NUCB2, PPARG, PTK2, H2BC12, APPBP2, ETV1, ERBB2, BNIP3L, TUBA1A, TGFB3, HMG20B, ALDH7A1, ACAT2, NIPAL3, ZNF33B, STMN1, DYNLT3, DNMT3A, EGF, CAST. [Provenance: CMAP/LINCS compound (trt_cp)]

